# Tradeoff between "sensor noise" and "process noise" for a scalar system

Do all imports:

In [1]:
# Stuff for computation
import numpy as np
from scipy import linalg
from scipy import signal

# Stuff for visualization
import ipywidgets as widgets
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column, row, Spacer
from bokeh.plotting import figure
from bokeh.models import Div

[Display Bokeh plots inline](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html#classic-notebooks):

In [2]:
output_notebook()

Loading BokehJS ...

Suppress the use of scientific notation when printing small numbers:

In [3]:
np.set_printoptions(suppress=True)

Define a function that returns a solution to the LQR problem.

In [4]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

Define a simulator class that can do two things:

* Run a simulator (`run_sim`) to generate the state $x$, input $u$, and output $y$, as well as disturbance $d$ and sensor noise $n$.
* Run an observer (`run_obs`) to compute the state estimate $\widehat{x}$ given the input $u$ and output $y$.

Recall that state estimates are computed by integrating the following ODE:

$$\dot{\widehat{x}} = A\widehat{x} + Bu - L(C\widehat{x} - y)$$

If `simulator.use_euler=True`, then the simulator class will apply Euler integration to compute state estimates as we often do when implementing continuous-time observers in practice.

If `simulator.use_euler=False`, then the simulator class will integrate the observer equation "exactly" (e.g., with a Runge-Kutta solver).

Note that, in either case, the simulator class shifts the state estimate forward by one time step so that, when we plot the results, we emphasize the association between the output (i.e., the sensor measurements) and the state estimates.

In [5]:
class Simulator:
    def __init__(self, t0=0., t1=5., dt=0.04, seed=None):
        self.t0 = t0
        self.t1 = t1
        self.dt = dt
        self.rng = np.random.default_rng(seed)
        
        self.A = np.array([[0.]])
        self.B = np.array([[1.]])
        self.C = np.array([[1.]])
        self.D = np.array([[0.]])
        
        self.x0 = np.array([1.])
        self.u_avg = 0.
        self.d_std = 0.1
        self.n_std = 0.1
        
        self.xhat0 = np.array([1.])
        self.L = np.array([[1.]])
        self.use_euler=False
        
        self.shift_time = True # <-- really shouldn't be here, since it has to do only with display
        
        self.t = np.linspace(t0, t1, 1 + np.ceil((t1 - t0) / dt).astype(int))
        self.nt = len(self.t)
    
    def run_sim(self):
        self.u = self.u_avg * np.ones((self.nt, 1))
        self.d = self.d_std * self.rng.standard_normal((self.nt, 1))
        self.n = self.n_std * self.rng.standard_normal((self.nt, 1))
        _, y, x = signal.lsim(
            (self.A, self.B, self.C, self.D),
            (self.u + self.d).flatten(),
            self.t,
            X0=self.x0,
            interp=False,
        )
        self.y = np.reshape(y, (-1, 1)) + self.n
        self.x = np.reshape(x, (-1, 1))
    
    def run_obs(self):
        if self.use_euler:
            xhat = np.empty((self.nt + 1, 1))
            xhat[0] = self.xhat0
            for i in range(self.nt):
                xhat[i + 1] = xhat[i] + self.dt * (self.A @ xhat[i] + self.B @ self.u[i] \
                                                   - self.L @ (self.C @ xhat[i] - self.y[i]))
            self.xhat = xhat[:-1, :]
            
        else:
            _, _, xhat = signal.lsim(
                (self.A - self.L @ self.C, np.hstack([self.B, self.L]), self.C, np.zeros((1, 2))),
                np.vstack([self.u.flatten(), self.y.flatten()]).T,
                self.t,
                X0=self.xhat0,
                interp=False,
            )
            self.xhat = np.reshape(xhat, (-1, 1))

Create an instance of the simulator.

In [6]:
simulator = Simulator()

Create interactive visualization.

In [7]:
# Widgets
solve_lqr = widgets.Checkbox(description='Solve LQR', value=True)
use_euler = widgets.Checkbox(description='Use Euler', value=simulator.use_euler)
shift_time = widgets.Checkbox(description='Plot xhat(t+dt)', value=True)
button = widgets.Button(description='Simulate')
x0s = widgets.FloatSlider(
    min=0,
    max=2,
    step=0.1,
    value=simulator.x0.item(),
    description='x0',
    readout_format='.3f',
    layout=widgets.Layout(width='auto'),
)
uavgs = widgets.FloatSlider(
    min=-1,
    max=1,
    step=0.1,
    value=simulator.u_avg,
    description='u',
    readout_format='.3f',
    layout=widgets.Layout(width='auto'),
)
dstds = widgets.FloatLogSlider(
    min=-2,
    max=2,
    step=0.1,
    value=simulator.d_std,
    description='std(d)',
    readout_format='.3f',
    layout=widgets.Layout(width='auto'),
)
nstds = widgets.FloatLogSlider(
    min=-2,
    max=2,
    step=0.1,
    value=simulator.n_std,
    description='std(n)',
    readout_format='.3f',
    layout=widgets.Layout(width='auto'),
)
xhat0s = widgets.FloatSlider(
    min=0,
    max=2,
    step=0.1,
    value=simulator.xhat0.item(),
    description='xhat0',
    readout_format='.3f',
    layout=widgets.Layout(width='auto'),
)
ls = widgets.FloatLogSlider(
    min=-5,
    max=5,
    step=0.1,
    value=simulator.L.item(),
    description='\u2113',
    layout=widgets.Layout(width='auto'),
)
qs = widgets.FloatLogSlider(
    min=-3,
    max=3,
    step=0.1,
    value=1.,
    description='q',
    layout=widgets.Layout(width='auto'),
)
rs = widgets.FloatLogSlider(
    min=-3,
    max=3,
    step=0.1,
    value=1,
    description='r',
    layout=widgets.Layout(width='auto'),
)

def run_obs():
    simulator.run_obs()
    if simulator.shift_time:
        x_plt_xhat.data_source.data['y'] = simulator.xhat.flatten()[1:]
    else:
        x_plt_xhat.data_source.data['y'] = simulator.xhat.flatten()[:-1]
    push_notebook()
    
def run_sim(button=None):
    simulator.run_sim()
    x_plt_x.data_source.data['y'] = simulator.x.flatten()
    x_plt_y.data_source.data['y'] = simulator.y.flatten()
    run_obs()

def update_sim_params(x0=1., uavg=0., dstd=1., nstd=1.):
    simulator.x0 = np.array([x0])
    simulator.u_avg = uavg
    simulator.d_std = dstd
    simulator.n_std = nstd

def update_obs_params(l=1., xhat0=1., q=1., r=1., solve_lqr=False, use_euler=False, shift_time=False):
    if solve_lqr:
        ls.disabled = True
        ls.continuous_update = False # <-- IMPORTANT! OTHERWISE YOU GET TORNADO WEBSOCKET ASSERTIONERRORS
                                     #     BECAUSE (I THINK) THE IPYWIDGETS ARE GOING BACK AND FORTH WITH
                                     #     EACH OTHER'S CALLBACKS (should revisit the option of using only
                                     #     bokeh widgets instead of ipywidgets)
        Qo = np.array([[q]])
        Ro = np.array([[r]])
        L = lqr(simulator.A.T, simulator.C.T, linalg.inv(Ro), linalg.inv(Qo)).T
        l = L[0, 0]
        ls.value = l
    else:
        L = np.array([[l]])
        ls.disabled = False
        ls.continuous_update = True
    simulator.L = L
    simulator.xhat0 = np.array([[xhat0]])
    simulator.use_euler = use_euler
    simulator.shift_time = shift_time
    run_obs()
    
# Plots
x_fig = figure(
    height=480,
    x_range=(simulator.t0, simulator.t1),
    y_range=(0, 2),
    x_axis_label='time (seconds)'
)
x_plt_x = x_fig.line(
    simulator.t,
    np.empty_like(simulator.t),
    line_width=4,
    line_color='navy',
    legend_label='state (x)',
)
x_plt_xhat = x_fig.line(
    simulator.t[0:-1],
    np.empty_like(simulator.t[0:-1]),
    line_width=2,
    line_color='coral',
    line_dash='solid',
    legend_label='state estimate (xhat)',
)
x_plt_y = x_fig.circle(
    simulator.t,
    np.empty_like(simulator.t),
    color='green',
    legend_label='output (y)',
    size=6,
)

# Layout (bokeh)
show(
    row(x_fig, sizing_mode='stretch_width'),
    notebook_handle=True,
)

# Run simulator and observer for the first time
run_sim()

# Layout (widgets)
ui = widgets.VBox([
    widgets.HBox([
        widgets.VBox([button], layout=widgets.Layout(width='20%')),
        widgets.VBox([x0s, uavgs], layout=widgets.Layout(width='40%')),
        widgets.VBox([dstds, nstds], layout=widgets.Layout(width='40%')),
    ], layout=widgets.Layout(border='solid 1px', width='100%')),
    widgets.HBox([
        widgets.VBox([solve_lqr, use_euler, shift_time], layout=widgets.Layout(width='20%')),
        widgets.VBox([ls, xhat0s], layout=widgets.Layout(width='30%')),
        widgets.VBox([qs, rs], layout=widgets.Layout(width='50%'))
    ], layout=widgets.Layout(border='solid 1px', width='100%'))
])

# Interactions (widgets)
button.on_click(run_sim)
out_state = widgets.interactive_output(
    update_sim_params,
    {
        'x0': x0s,
        'uavg': uavgs,
        'dstd': dstds,
        'nstd': nstds,
    }
)
out_estimate = widgets.interactive_output(
    update_obs_params,
    {
        'l': ls,
        'xhat0': xhat0s,
        'q': qs,
        'r': rs,
        'solve_lqr': solve_lqr,
        'use_euler': use_euler,
        'shift_time': shift_time,
    }
)

# Display (widgets)
display(ui, out_state, out_estimate)

Output()

Output()